In [206]:
# !python -m pip install geopandas
import pandas as pd
import numpy as np
import geopandas as gpd
from scipy.stats import zscore
import geopandas as gpd
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show
from bokeh.models import GeoJSONDataSource, ColumnDataSource, HoverTool
from bokeh.palettes import Viridis256, Inferno256, Magma256
from bokeh.palettes import BrBG, PiYG, RdGy, RdYlGn, YlGnBu
from bokeh.models import LinearColorMapper, ColorBar
import glob


In [207]:
import os

xls = pd.ExcelFile('../gaiins/data/ConsolidatedCountyCrops_V3.xlsx')
bananas = pd.read_excel(xls, 'Bananas')
barley = pd.read_excel(xls, 'Barley')
cabbages = pd.read_excel(xls, 'Cabbages')
cassava = pd.read_excel(xls, 'Cassava')
chickpeas = pd.read_excel(xls, 'chickpeas')
cocoyams = pd.read_excel(xls, 'cocoyams')
cowpeas = pd.read_excel(xls, 'CowPeas')
dolichos = pd.read_excel(xls, 'Dolichos')
beans = pd.read_excel(xls, 'DryBeans')
fingermillet = pd.read_excel(xls, 'FingerMillet')
gardenPeas = pd.read_excel(xls, 'GardenPeas')
grainamaranth = pd.read_excel(xls, 'GrainAmaranth')
gardenpeas = pd.read_excel(xls, 'GardenPeas')

kales = pd.read_excel(xls, 'Kales')
macadamia = pd.read_excel(xls, 'Macadamia')
maize = pd.read_excel(xls, 'Maize')
mangoes = pd.read_excel(xls, 'Mangoes')
millet = pd.read_excel(xls, 'Millet')
pearlmillet = pd.read_excel(xls, 'PearlMillet')
pigeonpeas = pd.read_excel(xls, 'PigeonPeas')
rice = pd.read_excel(xls, 'Rice')
simsim = pd.read_excel(xls, 'Simsim')
Sorghum = pd.read_excel(xls, 'Sorghum')
soyabeans = pd.read_excel(xls, 'SoyaBeans')
sweetpotatoes = pd.read_excel(xls, 'SweetPotatoes')
wheat = pd.read_excel(xls, 'Wheat')
yams = pd.read_excel(xls, 'Yams')


In [208]:
# concatenate the dataframes
crops=pd.concat([bananas,barley,cabbages,cassava,chickpeas,cocoyams,
                 cowpeas,dolichos,beans,fingermillet,gardenPeas,grainamaranth,
                 gardenpeas,kales,macadamia,maize,mangoes,millet,pearlmillet,pigeonpeas,
                 rice,simsim,Sorghum,soyabeans,sweetpotatoes,wheat,yams],ignore_index=True)
crops['county1']=crops['county']


In [209]:
# Set county1 names to lower case
crops.crop=crops.crop.str.lower()
crops.county1=crops.county1.str.lower()

crops.columns=crops.columns.str.lower()
crops.columns=crops.columns.str.strip()
crops.columns=crops.columns.str.replace(' ', '', regex=True)#remove space in between

crops.crop=crops.crop.str.lower()
crops.crop=crops.crop.str.strip()
crops.crop=crops.crop.str.replace(' ', '', regex=True)#remove space in between
crops.crop=crops.crop.str.replace('-', '', regex=True)#remove space in between

crops.county1=crops.county1.str.lower()
crops.county1=crops.county1.str.strip()
crops.county1=crops.county1.str.replace(' ', '', regex=True)#remove space in between
crops.county1=crops.county1.str.replace('-', '', regex=True)#remove space in between
# crops['yield2']=crops['production_mt']/crops['area_ha']

In [210]:
# read and clean shapefile
kenya_county=gpd.read_file('../kenya/data/shp/gadm41_KEN_1.shp')
kenya_county['county1']=kenya_county['NAME1']

kenya_county.columns=kenya_county.columns.str.lower()
kenya_county.columns=kenya_county.columns.str.strip()
kenya_county.columns=kenya_county.columns.str.replace(' ', '', regex=True)#remove space in between

kenya_county.county1=kenya_county.county1.str.lower()
kenya_county.county1=kenya_county.county1.str.strip()
kenya_county.county1=kenya_county.county1.str.replace(' ', '', regex=True)#remove space in between
kenya_county.county1=kenya_county.county1.str.replace('-', '', regex=True)#remove space in between
kenya_county.columns.unique()

crops.to_excel('data/gains_counties_data.xlsx',index=False)

kenya_county.columns.unique()
def clip_shp(data,shapefile):
    df_list=[]
    for i in list(data.county1.unique()):
        df=shapefile.loc[shapefile['county1']==i]
        df_list.append(df)
    df1=pd.concat(df_list,ignore_index=True)
    return df1
shapefile=clip_shp(crops,kenya_county)

In [229]:
# check if the data county names are same as in shapefile and extract counties
def crop_shapefile(data,shp,croptype):
    county_list=list(data.county1.unique())
    datalist=[]
    datalist2=[]
    datalist3=[]
    for county in list(data.county1.unique()):
        if county in list(shp.county1.unique()):
            datalist.append(county)
    for i in datalist:
        df1=data.loc[data['county1']==i]
        datalist2.append(df1)
    df5=pd.concat(datalist2, ignore_index=True)
    for countyname in county_list:
        df6=df5.loc[df5['county1']==countyname]
        datalist3.append(df6)
    df7=pd.concat(datalist3,ignore_index=True)
    df8=df7.loc[df7['crop']==croptype]
    df9=df8.groupby(['county1'],as_index=False).agg({'yield_mt_ha':'mean'})
    shapefile=pd.merge(kenya_county,df9, on='county1')
    return shapefile

def generate_crop_shapefile(data,shapefile):
    for i in list(data.crop.unique()):
        shapefile1=crop_shapefile(data=crops, shp=shapefile,croptype=i)
        shapefile1.to_file('../gaiins/data/crop_shapefiles/{0}.shp'.format(i))
# generate_crop_shapefile(crops,shapefile=shapefile)
cassava=gpd.read_file('../gaiins/data/crop_shapefiles/cassava.shp')
cassava

,gid1,gid_0,country,name1,varname_1,nl_name_1,type_1,engtype_1,cc_1,hasc_1,iso_1,county1,yield_mt_h,geometry
0,KEN.3_1,KEN,Kenya,Bungoma,NA,NA,County,County,39,KE.BN,KE-03,bungoma,16.765000,"POLYGON ((34.87783 0.83390, 34.88717 0.83884, ..."
1,KEN.6_1,KEN,Kenya,Embu,NA,NA,County,County,14,KE.EB,KE-06,embu,17.831667,"POLYGON ((37.67174 -0.82957, 37.67160 -0.82993..."
2,KEN.23_1,KEN,Kenya,Makueni,NA,NA,County,County,17,KE.MK,KE-23,makueni,9.345000,"POLYGON ((38.31715 -2.52309, 38.31716 -2.52328..."
3,KEN.31_1,KEN,Kenya,Nakuru,NA,NA,County,County,32,KE.NK,KE-31,nakuru,10.968333,"POLYGON ((36.52649 -1.14046, 36.52284 -1.13768..."
4,KEN.44_1,KEN,Kenya,Uasin Gishu,NA,NA,County,County,27,KE.UG,KE-44,uasingishu,9.616667,"POLYGON ((35.57412 0.06903, 35.57531 0.06567, ..."


In [178]:
geo_source_crop_shp = GeoJSONDataSource(geojson=shapefile.to_json())
country_geo_source = GeoJSONDataSource(geojson=kenya_county.to_json())

In [179]:
color_mapper = LinearColorMapper(palette="RdBu11", low=0.55, high=3.73)
p1 = figure(title='Maize Crop',
           outer_width=650, outer_height=600)

p1.multi_line('xs','ys', source=geo_source_crop_shp,
            line_color='black', line_width=0.5)

p1.patches('xs','ys', source=geo_source_crop_shp,
          fill_color={'field': 'yield_mt_ha', 'transform': color_mapper},
          fill_alpha=0.7, line_color='black', line_width=0.5)

hover = HoverTool(tooltips=[('District', '@name1'), ('yield', '@yield_mt_ha')])
p1.add_tools(hover)

color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8, width=20, height=300,
                     border_line_color=None, location=(0,0), orientation='vertical')
p1.add_layout(color_bar, 'right')

In [180]:
grid = gridplot([p1], ncols=1, width=650, height=600)
show(grid)